In [10]:
# Import Library
import joblib
import streamlit
# print(streamlit.__file__)
import sklearn_crfsuite
from faker import Faker
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
from folium import GeoJson
import mapclassify
import matplotlib.pyplot as plt
import contextily as ctx
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from ipywidgets import interactive
from IPython.display import display, HTML
%matplotlib inline
import plotly.express as px
import datetime
import math
import matplotlib.ticker as mtick
from numpy.polynomial.polynomial import polyfit

Load Model

In [11]:
model = joblib.load("model.joblib")

In [12]:
stopwords = ["ผู้", "ที่", "ซึ่ง", "อัน"]

def tokens_to_features(tokens, i):
  word = tokens[i]

  features = {
    "bias": 1.0,
    "word.word": word,
    "word[:3]": word[:3],
    "word.isspace()": word.isspace(),
    "word.is_stopword()": word in stopwords,
    "word.isdigit()": word.isdigit(),
    "word.islen5": word.isdigit() and len(word) == 5
  }

  if i > 0:
    prevword = tokens[i - 1]
    features.update({
      "-1.word.prevword": prevword,
      "-1.word.isspace()": prevword.isspace(),
      "-1.word.is_stopword()": prevword in stopwords,
      "-1.word.isdigit()": prevword.isdigit(),
    })
  else:
    features["BOS"] = True

  if i < len(tokens) - 1:
    nextword = tokens[i + 1]
    features.update({
      "+1.word.nextword": nextword,
      "+1.word.isspace()": nextword.isspace(),
      "+1.word.is_stopword()": nextword in stopwords,
      "+1.word.isdigit()": nextword.isdigit(),
    })
  else:
    features["EOS"] = True

  return features

# def parse(text):
#   tokens = text.split()
#   features = [tokens_to_features(tokens, i) for i in range(len(tokens))]

#   print(model.predict([features])[0])

def parse(text):
    if pd.notnull(text):
        tokens = text.split()
        features = [tokens_to_features(tokens, i) for i in range(len(tokens))]
        predicted_labels = model.predict([features])[0]
        return predicted_labels  # Always return the prediction labels
    else:
        return []  # Return an empty list if the input is invalid or empty


In [13]:
parse("นายสมชาย เข็มกลัด 254 ถนน พญาไท แขวง วังใหม่ เขต ปทุมวัน กรุงเทพมหานคร 10330")

array(['O', 'O', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'LOC',
       'LOC', 'POST'], dtype=object)

In [14]:
parse("นายมงคล 123/4 ตำบล บ้านไกล อำเภอ เมือง จังหวัด ลพบุรี 15000")

array(['ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR', 'ADDR',
       'POST'], dtype=object)

Data Set from Ploy

In [15]:
#Load Data

file_path_1 = r"correct_dataset2"

df1 = pd.read_csv(file_path_1)

In [16]:
pd.set_option('display.max_columns', None)  # Show all columns
display(df1.head())  # Or df to display the full DataFrame (depending on size)

,Unnamed: 0,index_column,province,district,subdistrict,zipcode,latitude,longitude,full_address,data_type,name,address_number,street,name_street,address_with_name,m_name,m_surname,m_address_number,m_street,m_subdistrict,m_district,m_province,m_zipcode,m_name_flag,m_surname_flag,m_address_number_flag,m_street_flag,m_subdistrict_flag,m_district_flag,m_province_flag,m_zipcode_flag
0,0,0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,พระบรมมหาราชวัง พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,รอซีด๊ะ ทรัพย์ธำรงค์,0,ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป พระบรมมหาราชวั...,O,O,ADDR,ADDR,ADDR,ADDR,LOC,POST,1,1,1,1,0,0,1,1
1,1,1,กรุงเทพมหานคร,พระนคร,วังบูรพาภิรมย์,10200,13.744,100.499,วังบูรพาภิรมย์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,พิมพ์พิชญา ดิสกะประกาย,7/6,พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล วังบูรพ...,LOC,LOC,LOC,LOC,LOC,LOC,LOC,POST,0,0,0,0,1,1,1,1
2,2,2,กรุงเทพมหานคร,พระนคร,วัดราชบพิธ,10200,13.750,100.499,วัดราชบพิธ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,อาซือมะ นาถะพินธุ,735/44,ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์ วัดราชบพิธ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1
3,3,3,กรุงเทพมหานคร,พระนคร,สำราญราษฎร์,10200,13.751,100.503,สำราญราษฎร์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,มะสูเกียน หอมสิน,857/7,เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช สำราญราษฎร์ พ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1
4,4,4,กรุงเทพมหานคร,พระนคร,ศาลเจ้าพ่อเสือ,10200,13.754,100.497,ศาลเจ้าพ่อเสือ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,ยูซุฟ ถนัดภาษา,745/4,ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน ศาลเจ้าพ่อเสือ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1


In [17]:
# Reset index and drop the first column by name
df1 = df1.reset_index(drop=True)  # This will reset the index and drop the old one
df1 = df1.drop(columns=['Unnamed: 0'])  # Drop the 'Unnamed: 0' column

In [18]:
#unique_id
df1['unique_id'] =  df1['province'].astype(str)+'_'+df1['district'].astype(str)+'_'+df1['subdistrict'].astype(str)

In [19]:
#Applly model to full_address column
# df1['test_model'] = df1['full_address'].apply(parse)
df1['test_model'] = df1['full_address'].apply(lambda x: parse(x) if pd.notnull(x) else [])

In [20]:
# display(df1['test_model'])
# print(df1['test_model'].iloc[0])  # Check the first value
# display(df1[['full_address', 'test_model']])
display(df1.head())

,index_column,province,district,subdistrict,zipcode,latitude,longitude,full_address,data_type,name,address_number,street,name_street,address_with_name,m_name,m_surname,m_address_number,m_street,m_subdistrict,m_district,m_province,m_zipcode,m_name_flag,m_surname_flag,m_address_number_flag,m_street_flag,m_subdistrict_flag,m_district_flag,m_province_flag,m_zipcode_flag,unique_id,test_model
0,0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,พระบรมมหาราชวัง พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,รอซีด๊ะ ทรัพย์ธำรงค์,0,ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป พระบรมมหาราชวั...,O,O,ADDR,ADDR,ADDR,ADDR,LOC,POST,1,1,1,1,0,0,1,1,กรุงเทพมหานคร_พระนคร_พระบรมมหาราชวัง,"[LOC, LOC, LOC, POST]"
1,1,กรุงเทพมหานคร,พระนคร,วังบูรพาภิรมย์,10200,13.744,100.499,วังบูรพาภิรมย์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,พิมพ์พิชญา ดิสกะประกาย,7/6,พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล วังบูรพ...,LOC,LOC,LOC,LOC,LOC,LOC,LOC,POST,0,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_วังบูรพาภิรมย์,"[LOC, LOC, LOC, POST]"
2,2,กรุงเทพมหานคร,พระนคร,วัดราชบพิธ,10200,13.750,100.499,วัดราชบพิธ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,อาซือมะ นาถะพินธุ,735/44,ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์ วัดราชบพิธ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_วัดราชบพิธ,"[LOC, LOC, LOC, POST]"
3,3,กรุงเทพมหานคร,พระนคร,สำราญราษฎร์,10200,13.751,100.503,สำราญราษฎร์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,มะสูเกียน หอมสิน,857/7,เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช สำราญราษฎร์ พ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_สำราญราษฎร์,"[LOC, LOC, LOC, POST]"
4,4,กรุงเทพมหานคร,พระนคร,ศาลเจ้าพ่อเสือ,10200,13.754,100.497,ศาลเจ้าพ่อเสือ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,ยูซุฟ ถนัดภาษา,745/4,ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน ศาลเจ้าพ่อเสือ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_ศาลเจ้าพ่อเสือ,"[LOC, LOC, LOC, POST]"


In [61]:
# df1.to_excel(r"C:\Users\User\OneDrive\Desktop\Chula Stat\Semester 1\Data Visualization\Project 3\filename.xlsx", index=False)

In [21]:
display(df1.head())

,index_column,province,district,subdistrict,zipcode,latitude,longitude,full_address,data_type,name,address_number,street,name_street,address_with_name,m_name,m_surname,m_address_number,m_street,m_subdistrict,m_district,m_province,m_zipcode,m_name_flag,m_surname_flag,m_address_number_flag,m_street_flag,m_subdistrict_flag,m_district_flag,m_province_flag,m_zipcode_flag,unique_id,test_model
0,0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,พระบรมมหาราชวัง พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,รอซีด๊ะ ทรัพย์ธำรงค์,0,ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป พระบรมมหาราชวั...,O,O,ADDR,ADDR,ADDR,ADDR,LOC,POST,1,1,1,1,0,0,1,1,กรุงเทพมหานคร_พระนคร_พระบรมมหาราชวัง,"[LOC, LOC, LOC, POST]"
1,1,กรุงเทพมหานคร,พระนคร,วังบูรพาภิรมย์,10200,13.744,100.499,วังบูรพาภิรมย์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,พิมพ์พิชญา ดิสกะประกาย,7/6,พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล วังบูรพ...,LOC,LOC,LOC,LOC,LOC,LOC,LOC,POST,0,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_วังบูรพาภิรมย์,"[LOC, LOC, LOC, POST]"
2,2,กรุงเทพมหานคร,พระนคร,วัดราชบพิธ,10200,13.750,100.499,วัดราชบพิธ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,อาซือมะ นาถะพินธุ,735/44,ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์,อาซือมะ นาถะพินธุ 735/44 ธุวะนุติ์ วัดราชบพิธ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_วัดราชบพิธ,"[LOC, LOC, LOC, POST]"
3,3,กรุงเทพมหานคร,พระนคร,สำราญราษฎร์,10200,13.751,100.503,สำราญราษฎร์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,มะสูเกียน หอมสิน,857/7,เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช,มะสูเกียน หอมสิน 857/7 เตมิยะเดช สำราญราษฎร์ พ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_สำราญราษฎร์,"[LOC, LOC, LOC, POST]"
4,4,กรุงเทพมหานคร,พระนคร,ศาลเจ้าพ่อเสือ,10200,13.754,100.497,ศาลเจ้าพ่อเสือ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,ยูซุฟ ถนัดภาษา,745/4,ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน,ยูซุฟ ถนัดภาษา 745/4 ซ.ดาบเงิน ศาลเจ้าพ่อเสือ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1,0,0,0,1,1,1,1,กรุงเทพมหานคร_พระนคร_ศาลเจ้าพ่อเสือ,"[LOC, LOC, LOC, POST]"


Thailand Map SHP Data Download & Cleaning

In [22]:
thailand_shape_amphoe_tambon = gpd.read_file(r"thailand_province_amphoe_tambon_simplify")


#source: https://data.thailand.opendevelopmentmekong.net/th/dataset/thailand-provincial-boundaries?type=dataset
# https://github.com/prasertcbs/thailand_gis

In [23]:
print(thailand_shape_amphoe_tambon.columns)

thailand_shape_amphoe_tambon

Index(['ADM3_EN', 'ADM3_TH', 'ADM3_PCODE', 'ADM2_EN', 'ADM2_TH', 'ADM2_PCODE',
       'ADM1_EN', 'ADM1_TH', 'ADM1_PCODE', 'ADM0_EN', 'ADM0_TH', 'ADM0_PCODE',
       'geometry'],
      dtype='object')


,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757..."
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.748, 100.50412 13.74771..."
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.748, 100.49633 13.74849..."
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50554 13.75378, 100.50412 13.747..."
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49875 13.7556, 100.49821 13.7514..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7420,Phadung Mat,ผดุงมาตร,TH961203,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.71286 6.17046, 101.71403 6.16538..."
7421,Chang Phueak,ช้างเผือก,TH961204,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.67997 5.99753, 101.67567 5.98895..."
7422,Chuap,จวบ,TH961301,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.86883 6.29824, 101.86908 6.29436..."
7423,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.8857 6.19545, 101.87473 6.19872,..."


In [24]:
#rename columns
thailand_shape_amphoe_tambon = thailand_shape_amphoe_tambon.rename(columns={
    'ADM3_EN': 'Tambon (EN)'
    ,'ADM3_TH': 'Tambon (TH)'
    ,'ADM3_PCODE': 'Tambon_PCODE'
    ,'ADM2_EN': 'Amphoe (EN)'
    ,'ADM2_TH': 'Amphoe (TH)'
    ,'ADM2_PCODE': 'Amphoe_PCODE'
    ,'ADM1_EN': 'Province (EN)'
    ,'ADM1_TH': 'Province (TH)'
    ,'ADM1_PCODE': 'Province_PCODE'
    ,'ADM0_EN': 'Country (EN)'
    ,'ADM0_TH': 'Country (TH)'
    ,'ADM0_PCODE': 'Country_PCODE'
})

In [25]:
display(thailand_shape_amphoe_tambon.columns)
column_dropdown = ['Tambon (TH)', 'Amphoe (TH)', 'Province (TH)']
# display(thailand_shape_amphoe_tambon.head())
display(column_dropdown)

Index(['Tambon (EN)', 'Tambon (TH)', 'Tambon_PCODE', 'Amphoe (EN)',
       'Amphoe (TH)', 'Amphoe_PCODE', 'Province (EN)', 'Province (TH)',
       'Province_PCODE', 'Country (EN)', 'Country (TH)', 'Country_PCODE',
       'geometry'],
      dtype='object')

['Tambon (TH)', 'Amphoe (TH)', 'Province (TH)']

In [26]:
#unique_id
thailand_shape_amphoe_tambon['unique_id'] =  thailand_shape_amphoe_tambon['Province (TH)'].astype(str)+'_'+thailand_shape_amphoe_tambon['Amphoe (TH)'].astype(str)+'_'+thailand_shape_amphoe_tambon['Tambon (TH)'].astype(str)

In [27]:
display(df1.columns)

display(thailand_shape_amphoe_tambon.columns)

Index(['index_column', 'province', 'district', 'subdistrict', 'zipcode',
       'latitude', 'longitude', 'full_address', 'data_type', 'name',
       'address_number', 'street', 'name_street', 'address_with_name',
       'm_name', 'm_surname', 'm_address_number', 'm_street', 'm_subdistrict',
       'm_district', 'm_province', 'm_zipcode', 'm_name_flag',
       'm_surname_flag', 'm_address_number_flag', 'm_street_flag',
       'm_subdistrict_flag', 'm_district_flag', 'm_province_flag',
       'm_zipcode_flag', 'unique_id', 'test_model'],
      dtype='object')

Index(['Tambon (EN)', 'Tambon (TH)', 'Tambon_PCODE', 'Amphoe (EN)',
       'Amphoe (TH)', 'Amphoe_PCODE', 'Province (EN)', 'Province (TH)',
       'Province_PCODE', 'Country (EN)', 'Country (TH)', 'Country_PCODE',
       'geometry', 'unique_id'],
      dtype='object')

In [28]:
# Find unique values in both columns
unique_df1 = df1['unique_id'].unique()
unique_shape = thailand_shape_amphoe_tambon['unique_id'].unique()

# Identify non-matching values in each DataFrame
non_matching_in_df1 = set(unique_df1) - set(unique_shape)
non_matching_in_shape = set(unique_shape) - set(unique_df1)

# Count the number of non-matching values
num_non_matching_df1 = len(non_matching_in_df1)
num_non_matching_shape = len(non_matching_in_shape)

# Calculate matching values
matching_records = len(set(unique_df1) & set(unique_shape))

# Output results
print(f"Non-matching in df1: {num_non_matching_df1}")
print(f"Non-matching in thailand_shape_amphoe_tambon: {num_non_matching_shape}")
print(f"Matching records: {matching_records}")


Non-matching in df1: 7479
Non-matching in thailand_shape_amphoe_tambon: 51
Matching records: 7373


In [117]:
#Have to clean unmatched data
# non_matching_in_df1
# non_matching_in_shape

In [29]:
# Perform a left join on the 'unique_id' column
merged_df = thailand_shape_amphoe_tambon.merge(df1, on='unique_id', how='left')

display(merged_df)

merged_df.columns

,Tambon (EN),Tambon (TH),Tambon_PCODE,Amphoe (EN),Amphoe (TH),Amphoe_PCODE,Province (EN),Province (TH),Province_PCODE,Country (EN),Country (TH),Country_PCODE,geometry,unique_id,index_column,province,district,subdistrict,zipcode,latitude,longitude,full_address,data_type,name,address_number,street,name_street,address_with_name,m_name,m_surname,m_address_number,m_street,m_subdistrict,m_district,m_province,m_zipcode,m_name_flag,m_surname_flag,m_address_number_flag,m_street_flag,m_subdistrict_flag,m_district_flag,m_province_flag,m_zipcode_flag,test_model
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757...",กรุงเทพมหานคร_พระนคร_พระบรมมหาราชวัง,0.0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,พระบรมมหาราชวัง พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,รอซีด๊ะ ทรัพย์ธำรงค์,0,ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป,รอซีด๊ะ ทรัพย์ธำรงค์ 0 ธนประทีป พระบรมมหาราชวั...,O,O,ADDR,ADDR,ADDR,ADDR,LOC,POST,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,"[LOC, LOC, LOC, POST]"
1,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757...",กรุงเทพมหานคร_พระนคร_พระบรมมหาราชวัง,7426.0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,พระบรมมหาราชวัง พระนคร กรุงเทพมหานคร 10200,Correct with Short Prefix,จิม เช้าวันดี,166,นาคพันธุ์,จิม เช้าวันดี 166 นาคพันธุ์,จิม เช้าวันดี 166 นาคพันธุ์ พระบรมมหาราชวัง พร...,O,O,ADDR,ADDR,ADDR,ADDR,LOC,POST,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,"[LOC, LOC, LOC, POST]"
2,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757...",กรุงเทพมหานคร_พระนคร_พระบรมมหาราชวัง,14852.0,กรุงเทพมหานคร,พระนคร,พระบรมมหาราชวัง,10200,13.751,100.492,แขวงพระบรมมหาราชวัง เขตพระนคร กรุงเทพมหานคร 10200,Correct with Full Prefix,บุญเกิด ดุษฎีวนิช,68หมู่3,นำธวัช,บุญเกิด ดุษฎีวนิช 68หมู่3 นำธวัช,บุญเกิด ดุษฎีวนิช 68หมู่3 นำธวัช แขวงพระบรมมหา...,ADDR,ADDR,ADDR,ADDR,ADDR,ADDR,LOC,POST,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,"[O, O, LOC, POST]"
3,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.748, 100.50412 13.74771...",กรุงเทพมหานคร_พระนคร_วังบูรพาภิรมย์,1.0,กรุงเทพมหานคร,พระนคร,วังบูรพาภิรมย์,10200,13.744,100.499,วังบูรพาภิรมย์ พระนคร กรุงเทพมหานคร 10200,Correct without Prefix,พิมพ์พิชญา ดิสกะประกาย,7/6,พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล,พิมพ์พิชญา ดิสกะประกาย 7/6 พีระเพ็ญกุล วังบูรพ...,LOC,LOC,LOC,LOC,LOC,LOC,LOC,POST,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,"[LOC, LOC, LOC, POST]"
4,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.748, 100.50412 13.74771...",กรุงเทพมหานคร_พระนคร_วังบูรพาภิรมย์,7427.0,กรุงเทพมหานคร,พระนคร,วังบูรพาภิรมย์,10200,13.744,100.499,วังบูรพาภิรมย์ พระนคร กรุงเทพมหานคร 10200,Correct with Short Prefix,สมปอง นิยมสำหรวจ,487/3,นาฏคายี,สมปอง นิยมสำหรวจ 487/3 นาฏคายี,สมปอง นิยมสำหรวจ 487/3 นาฏคายี วังบูรพาภิรมย์ ...,O,LOC,LOC,LOC,LOC,LOC,LOC,POST,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,"[LOC, LOC, LOC, POST]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22166,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.8857 6.19545, 101.87473 6.19872,...",นราธิวาส_เจาะไอร้อง_บูกิต,14850.0,นราธิวาส,เจาะไอร้อง,บูกิต,96130,6.182,101.828,ต.บูกิต อ.เจาะไอร้อง นราธิวาส 96130,Correct with Short Prefix,นิรุตต์ ถ้วนศรี,91หมู่08,ขอหมั่นกลาง,นิรุตต์ ถ้วนศรี 91หมู่08 ขอหมั่นกลาง,นิรุตต์ ถ้วนศรี 91หมู่08 ขอหมั่นกลาง ต.บูกิต อ...,O,O,O,O,O,O,O,POST,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,"[O, O, O, POST]"
22167,Bu Kit,บูกิต,TH

Index(['Tambon (EN)', 'Tambon (TH)', 'Tambon_PCODE', 'Amphoe (EN)',
       'Amphoe (TH)', 'Amphoe_PCODE', 'Province (EN)', 'Province (TH)',
       'Province_PCODE', 'Country (EN)', 'Country (TH)', 'Country_PCODE',
       'geometry', 'unique_id', 'index_column', 'province', 'district',
       'subdistrict', 'zipcode', 'latitude', 'longitude', 'full_address',
       'data_type', 'name', 'address_number', 'street', 'name_street',
       'address_with_name', 'm_name', 'm_surname', 'm_address_number',
       'm_street', 'm_subdistrict', 'm_district', 'm_province', 'm_zipcode',
       'm_name_flag', 'm_surname_flag', 'm_address_number_flag',
       'm_street_flag', 'm_subdistrict_flag', 'm_district_flag',
       'm_province_flag', 'm_zipcode_flag', 'test_model'],
      dtype='object')

In [30]:
thailand_shape_amphoe_tambon.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [31]:
merged_df['data_type'].unique()

array(['Correct without Prefix', 'Correct with Short Prefix',
       'Correct with Full Prefix', nan], dtype=object)

In [ ]:
# merged_df.to_excel(r"C:\Users\User\OneDrive\Desktop\Chula Stat\Semester 1\Data Visualization\Project 3\merged_df_2.xlsx", index=False)
# merged_df.to_csv(r"C:\Users\User\OneDrive\Desktop\Chula Stat\Semester 1\Data Visualization\Project 3\merged_df_2.csv", index=False)

Viz

In [33]:
def create_filtering_widgets():
    # Define the columns and their display names for entity checkboxes
    flag_columns = ['m_subdistrict_flag', 'm_district_flag', 'm_province_flag', 'm_zipcode_flag']
    flag_column_labels = {col: col.split('_')[1].capitalize() for col in flag_columns}
    
    # Define data type options
    data_type_options = merged_df['data_type'].dropna().unique() #dropna() to handle null values

    # Define grouping options
    grouping_options = [
        'No Grouping',
        'Province',
        'District',
        'Subdistrict',
        'Postal Code'
    ]

    # Create entity checkboxes
    entity_checkboxes = [
        widgets.Checkbox(
            value=True,
            description=display_name,
            layout=widgets.Layout(margin='5px'),
            style={'description_width': 'initial'}
        )
        for _, display_name in flag_column_labels.items()
    ]

    # Create data type checkboxes
    select_all_data_type = widgets.Checkbox(
        value=True,
        description='Select All Data Type',
        layout=widgets.Layout(margin='5px'),
        style={'description_width': 'initial'}
    )

    data_type_checkboxes = [
        widgets.Checkbox(
            value=True,
            description=option,
            layout=widgets.Layout(margin='5px'),
            style={'description_width': 'initial'}
        )
        for option in data_type_options
    ]

    # Create grouping dropdown
    grouping_dropdown = widgets.Dropdown(
        options=grouping_options,
        value='No Grouping',
        description='Group by:',
        layout=widgets.Layout(margin='5px', width='200px'),
        style={'description_width': 'initial'}
    )

    # Create map coloring dropdown ###torevist definition of this, to remove???? and combine with grouping dropdown!
    map_dropdown = widgets.Dropdown(
        options=list(column_dropdown),
        value='Province (TH)',
        description='Color by:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(margin='5px', width='200px')
    )


    return {
        'entity_checkboxes': entity_checkboxes,
        'select_all_data_type': select_all_data_type,
        'data_type_checkboxes': data_type_checkboxes,
        'grouping_dropdown': grouping_dropdown,
        'map_dropdown': map_dropdown,
        'flag_columns': flag_columns,
        'flag_column_labels': flag_column_labels
    }


def create_containers(widgets_dict):
    # Create containers for filtering controls
    entity_container = widgets.VBox([
        widgets.HTML(value="<h4>Select entities to include in recall calculation:</h4>"),
        widgets.VBox(widgets_dict['entity_checkboxes'], 
                    layout=widgets.Layout(
                        border='1px solid #ddd',
                        padding='10px',
                        margin='5px',
                        width='300px'
                    ))
    ])
    
    data_type_container = widgets.VBox([
        widgets.HTML(value="<h4>Filter by Data Type:</h4>"),
        widgets_dict['select_all_data_type'],
        widgets.VBox(widgets_dict['data_type_checkboxes'],
                    layout=widgets.Layout(
                        border='1px solid #ddd',
                        padding='10px',
                        margin='5px',
                        width='300px'
                    ))
    ])
    
    grouping_container = widgets.VBox([
        widgets.HTML(value="<h4>Grouping Options:</h4>"),
        widgets_dict['grouping_dropdown'],
        widgets.HTML(value="<h4>Map Coloring:</h4>"),
        widgets_dict['map_dropdown']
    ], layout=widgets.Layout(
        border='1px solid #ddd',
        padding='10px',
        margin='5px',
        width='300px'
    ))
    
    return entity_container, data_type_container, grouping_container

In [34]:
def calculate_group_metrics(df, group_col):
    """Calculate metrics for grouped data"""
    group_metrics = df.groupby(group_col).agg({
        'recall_per_row': ['count', 'mean', 'median', 'std'],
    }).round(4)
    
    group_metrics.columns = ['Count', 'Mean Recall', 'Median Recall', 'Std Dev']
    return group_metrics

In [35]:
def make_map(selected_column, filtered_df=None):
    """Create map with the selected column and filtered data"""
    base_map = thailand_shape_amphoe_tambon.explore(
        location=[13.7563, 100.5018],
        zoom_start=6.1,
        column=selected_column,
        tooltip=['Tambon (EN)', 'Tambon (TH)', 'Tambon_PCODE', 'Amphoe (EN)',
                'Amphoe (TH)', 'Amphoe_PCODE', 'Province (EN)', 'Province (TH)',
                'Province_PCODE', 'Country (EN)', 'Country (TH)', 'Country_PCODE'],
        popup=True,
        tiles='cartodbdark_matter',
        # tiles='CartoDB positron'
        # tiles='OpenStreetMap',
        style_kwds={
            'fill': True,  # Disable fill
            # 'color': 'white',  # Set border color to black
            'weight': 2,  # Set border thickness
            'opacity': 0.6
        },
        # style_kwds={'fillOpacity': 0.7},
        legend=True
    )
    
    # If filtered data is provided, update the map
    if filtered_df is not None:
        # Add filtered data visualization here
        # This will depend on your specific requirements
        pass
    
    return base_map


In [36]:
def create_output_widgets():
    """Create output widgets for metrics and map"""
    metrics_output = widgets.Output()
    map_output = widgets.Output()
    return metrics_output, map_output

In [37]:
def update_display(widgets_dict, metrics_output, map_output):
    """Update both metrics and map based on current selections"""
    
    def update(*args):
        with metrics_output:
            metrics_output.clear_output()
            
            # Get selected entities
            selected_columns = [
                list(widgets_dict['flag_column_labels'].keys())[i] 
                for i, checkbox in enumerate(widgets_dict['entity_checkboxes']) 
                if checkbox.value
            ]
            
            if not selected_columns:
                print("Please select at least one entity type")
                return
            
            # Get selected data types
            selected_data_types = [
                option.description
                for option in widgets_dict['data_type_checkboxes']
                if option.value
            ]
            
            # Apply filters
            filtered_df = merged_df.copy()
            if selected_data_types:
                filtered_df = filtered_df[filtered_df['data_type'].isin(selected_data_types)]
            
            # Calculate recall on filtered data
            filtered_df['recall_per_row'] = (filtered_df[selected_columns].sum(axis=1) / 
                                           filtered_df[selected_columns].notnull().sum(axis=1))
            
            # Handle grouping and display metrics
            grouping_level = widgets_dict['grouping_dropdown'].value
            if grouping_level != 'No Grouping':
                group_mapping = {
                    'Province': 'province',
                    'District': 'district',
                    'Subdistrict': 'subdistrict',
                    'Postal Code': 'zipcode'
                }
                
                group_col = group_mapping[grouping_level]
                group_metrics = calculate_group_metrics(filtered_df, group_col)
                
                print(f"\n\033[1mRecall Statistics Grouped by {grouping_level}:\033[0m")
                display(HTML(group_metrics.to_html()))
            else:
                print("\n\033[1mOverall Recall Statistics:\033[0m")
                print(f"Number of records: {len(filtered_df)}")
                print(f"Mean Recall: {filtered_df['recall_per_row'].mean():.4f}")
                print(f"Median Recall: {filtered_df['recall_per_row'].median():.4f}")
                print(f"Std Recall: {filtered_df['recall_per_row'].std():.4f}")
        
        # Update map
        with map_output:
            map_output.clear_output()
            display(make_map(widgets_dict['map_dropdown'].value, filtered_df))
    
    return update

In [38]:
def select_all_data_type_handler(change):
    """Handle select all checkbox for data types"""
    for checkbox in widgets_dict['data_type_checkboxes']:
        checkbox.value = change.new

In [39]:
def main():
    # Create all widgets
    widgets_dict = create_filtering_widgets()
    
    # Create containers
    entity_container, data_type_container, grouping_container = create_containers(widgets_dict)
    
    # Create output widgets
    metrics_output, map_output = create_output_widgets()
    
    # Create update function
    update = update_display(widgets_dict, metrics_output, map_output)
    
    # Connect all widgets to update function
    for checkbox in widgets_dict['entity_checkboxes']:
        checkbox.observe(update, names='value')
    
    for checkbox in widgets_dict['data_type_checkboxes']:
        checkbox.observe(update, names='value')
    
    widgets_dict['select_all_data_type'].observe(select_all_data_type_handler, names='value')
    widgets_dict['select_all_data_type'].observe(update, names='value')
    widgets_dict['grouping_dropdown'].observe(update, names='value')
    widgets_dict['map_dropdown'].observe(update, names='value')
    
    # Set the layout sizes for each widget
    entity_container.layout.width = '400px'
    data_type_container.layout.width = '400px'
    grouping_container.layout.width = '400px'
    metrics_output.layout.width = '500px'
    map_output.layout = widgets.Layout(width='600px', height='800px')
      
    
    # Create main container
    main_container = widgets.HBox([
        widgets.VBox([entity_container, data_type_container, grouping_container], 
                      layout=widgets.Layout(width='500px')),
        metrics_output,
        map_output
    ], layout=widgets.Layout(width='1500px'))
    
    # Initial calculation
    update()
    
    # Display the widget
    display(main_container)

In [40]:
# Run the application
if __name__ == "__main__":
    main()

DRAFT